In [4]:
# ===========================================
# Import de librerías para configuración de ambiente
import os
import random
print ("Random number with seed 73")
random.seed(73)

os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf_device='/gpu:0'
# ===========================================
# Import de qutip para simulación de la dinámica de sistemas cuánticos abiertos
import qutip as qt
# ===========================================
# Import de librerías para manejo de conjuntos de datos y operaciones matriciales
import pandas as pd

import numpy as np
from numpy.linalg import cholesky

from typing import Tuple, List

# ===========================================
# Función para generación de n muestras de estados aleatorios de 2 qubits

def xy2qb(random_state_dm: qt.Qobj, desc: int) -> Tuple[List[float], List[float]]:
    """
    Aplica descomposición de Cholesky opcionalmente sobre una matriz de densidad cuántica y 
    retorna una muestra de valores de la matriz junto con su concurrencia.

    Parámetros:
    ----------
    random_state_dm : qt.Qobj
        Objeto de matriz de densidad cuántica de QuTiP.
    
    desc : int
        Indicador para aplicar la descomposición de Cholesky:
        - Si es 0: no se aplica la descomposición de Cholesky.
        - Si es 1: se aplica la descomposición de Cholesky.

    Retorna:
    --------
    Tuple[List[float], List[float]]
        - Lista de valores reales e imaginarios de la matriz de densidad cuántica.
        - Lista con la concurrencia de la matriz de densidad cuántica.
    
    Excepciones:
    ------------
    - [Error] - Si el valor de `desc` no es válido (0 o 1).
    """
    
    p_trc_0 = random_state_dm.ptrace(0)
    p_trc_1 = random_state_dm.ptrace(1)
    
    if (desc == 0):
        #random_state_dm_arr = np.asarray(random_state_dm)
        random_state_dm_arr = random_state_dm.full()
        
        diag_elements = random_state_dm_arr.diagonal()
        decomp_rsdm_lower = random_state_dm_arr[np.tril_indices(4, k=-1)]
        decomp_rsdm_lower = decomp_rsdm_lower.flatten()
        
        p_trc_0_dm = p_trc_0.full()
        p_trc_1_dm = p_trc_1.full()
        
    elif (desc == 1):
        decomp_rsdm = cholesky(random_state_dm.full())
        diag_elements = decomp_rsdm.diagonal()
        decomp_rsdm_lower = decomp_rsdm[np.tril_indices(4, k=-1)]
        decomp_rsdm_lower = decomp_rsdm_lower.flatten()
        
        p_trc_0_dm = cholesky(p_trc_0.full())
        p_trc_1_dm = cholesky(p_trc_1.full())
    else:
        print('[Error] - Invalid value')
        return [], []
        
    d_ptrc_0 = p_trc_0_dm.diagonal()
    d_ptrc_1 = p_trc_1_dm.diagonal()
    d_ptrc_q0_lower = p_trc_0_dm[np.tril_indices(2, k=-1)]
    d_ptrc_q0_lower = d_ptrc_q0_lower.flatten()
    d_ptrc_q1_lower = p_trc_1_dm[np.tril_indices(2, k=-1)]
    d_ptrc_q1_lower = d_ptrc_q1_lower.flatten()
    
    x_sample = []

    for x in diag_elements:
        x_sample.append(x.real)
    for x in decomp_rsdm_lower:
        x_sample.append(x.real)
    for x in decomp_rsdm_lower:
        x_sample.append(x.imag)
                
    for y in d_ptrc_0:
        x_sample.append(y.real)
    for y in d_ptrc_q0_lower:
        x_sample.append(y.real)
    for y in d_ptrc_q0_lower:
        x_sample.append(y.imag)
        
    for z in d_ptrc_1:
        x_sample.append(z.real)
    for z in d_ptrc_q1_lower:
        x_sample.append(z.real)
    for z in d_ptrc_q1_lower:
        x_sample.append(z.imag)
                        
    conc = qt.concurrence(random_state_dm)
    
    return x_sample, [conc]

# ===========================================
# Proceso principal de generación de estados

n_samples = 10 # Definición de cantidad de muestras

columns = ["a11", "a22", "a33", "a44",
           "a21r", "a31r", "a32r", "a41r", "a42r", "a43r",
           "a21i", "a31i", "a32i", "a41i", "a42i", "a43i",
           "ptq0_11", "ptq0_22",
           "ptq0_21r", "ptq0_21i",
           "ptq1_11", "ptq1_22",
           "ptq1_21r", "ptq1_21i",
           "concurrence"]
#

rs_df_nodesc = pd.DataFrame(columns = columns) # Se crea el dataframe de salida con las columnas definidas
rs_df_desc = pd.DataFrame(columns = columns) # Se crea el dataframe de salida con las columnas definidas

# Para acumular las filas a agregar
rows_nodesc = []
rows_desc = []

for x in range(n_samples):
    # Genera una matriz de densidad cuántica aleatoria con distribución Ginibre
    random_state_dm = qt.rand_dm(distribution='ginibre', dimensions=[[2, 2]])
    
    # Sin descomposición
    x_sample,y_sample = xy2qb(random_state_dm, 0)
    xy_sample = x_sample+y_sample
    rows_nodesc.append(pd.Series(xy_sample, index=rs_df_nodesc.columns))
    
    # Con descomposición de Cholesky
    x_sample, y_sample = xy2qb(random_state_dm, 1)
    xy_sample = x_sample + y_sample
    rows_desc.append(pd.Series(xy_sample, index=rs_df_desc.columns))
        
# Agrega todas las filas al DataFrame al final
rs_df_nodesc = pd.concat([rs_df_nodesc] + rows_nodesc, ignore_index=True)
rs_df_desc = pd.concat([rs_df_desc] + rows_desc, ignore_index=True)

Random number with seed 73


In [6]:
rs_df_desc

,a11,a22,a33,a44,a21r,a31r,a32r,a41r,a42r,a43r,...,ptq0_11,ptq0_22,ptq0_21r,ptq0_21i,ptq1_11,ptq1_22,ptq1_21r,ptq1_21i,concurrence,0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276947
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.647853
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.251975
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.046034
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.841202
246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.528755
247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.049281
248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.101829


: 

In [ ]:
random_state_dm = qt.rand_dm(distribution='ginibre', dimensions=[[2, 2]])

In [ ]:
# ===========================================
# Guardado de conjuntos de datos

rs_df_nodesc.to_csv('./datasets/2qb_random_state_nodesc.csv', index=False, index_label=False)
rs_df_desc.to_csv('./datasets/2qb_random_state_desc.csv', index=False, index_label=False)

In [7]:
random_state_dm = qt.rand_dm(distribution='ginibre', dimensions=[[2, 2]])
random_state_dm

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.22598889+0.j         -0.03053797-0.1126419j   0.03389186+0.04573521j
   0.08458516+0.01126264j]
 [-0.03053797+0.1126419j   0.20736909+0.j          0.19066428-0.00588423j
   0.00911015-0.04488184j]
 [ 0.03389186-0.04573521j  0.19066428+0.00588423j  0.36472799+0.j
   0.02059084-0.13575365j]
 [ 0.08458516-0.01126264j  0.00911015+0.04488184j  0.02059084+0.13575365j
   0.20191403+0.j        ]]

In [11]:
random_state_dm_arr = np.array(random_state_dm)
random_state_dm_arr

array(Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.1525465 +0.j         -0.15200492+0.00665294j -0.01673195+0.01126816j
   0.17713204+0.04794142j]
 [-0.15200492-0.00665294j  0.19365002+0.j          0.00323593+0.03504698j
  -0.27500439-0.07637025j]
 [-0.01673195-0.01126816j  0.00323593-0.03504698j  0.13243541+0.j
  -0.002434  +0.05391672j]
 [ 0.17713204-0.04794142j -0.27500439+0.07637025j -0.002434  -0.05391672j
   0.52136808+0.j        ]], dtype=object)

In [12]:
diag_elements = random_state_dm_arr.diagonal()

ValueError: diag requires an array of at least two dimensions

In [16]:
random_state_dm = qt.rand_dm(distribution='ginibre', dimensions=[[2, 2]])
random_state_dm

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.18675617+0.j         -0.04439905+0.07815698j -0.02898772+0.18623514j
  -0.10752515-0.10141357j]
 [-0.04439905-0.07815698j  0.15629032+0.j          0.08636458-0.06907675j
  -0.05160792+0.05739769j]
 [-0.02898772-0.18623514j  0.08636458+0.06907675j  0.3602529 +0.j
  -0.13365637+0.2649671j ]
 [-0.10752515+0.10141357j -0.05160792-0.05739769j -0.13365637-0.2649671j
   0.29670061+0.j        ]]

In [17]:
random_state_dm_arr = random_state_dm.full()
random_state_dm_arr

array([[ 0.18675617+0.j        , -0.04439905+0.07815698j,
        -0.02898772+0.18623514j, -0.10752515-0.10141357j],
       [-0.04439905-0.07815698j,  0.15629032+0.j        ,
         0.08636458-0.06907675j, -0.05160792+0.05739769j],
       [-0.02898772-0.18623514j,  0.08636458+0.06907675j,
         0.3602529 +0.j        , -0.13365637+0.2649671j ],
       [-0.10752515+0.10141357j, -0.05160792-0.05739769j,
        -0.13365637-0.2649671j ,  0.29670061+0.j        ]])

In [18]:
diag_elements = random_state_dm_arr.diagonal()
diag_elements

array([0.18675617+0.j, 0.15629032+0.j, 0.3602529 +0.j, 0.29670061+0.j])

# Mezclados

In [ ]:
# ===========================================
# Import de librerías para configuración de ambiente
import os
import random
print ("Random number with seed 73")
random.seed(73)

os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf_device='/gpu:0'
# ===========================================
# Import de qutip para simulación de la dinámica de sistemas cuánticos abiertos
import qutip as qt
# ===========================================
# Import de librerías para manejo de conjuntos de datos y operaciones matriciales
import pandas as pd

import numpy as np
from numpy.linalg import cholesky

from typing import Tuple, List

# ===========================================
# Función para generación de n muestras de estados aleatorios de 2 qubits

def xy2qb(random_state_dm: qt.Qobj, desc: int) -> Tuple[List[float], List[float]]:
    """
    Aplica descomposición de Cholesky opcionalmente sobre una matriz de densidad cuántica y 
    retorna una muestra de valores de la matriz junto con su concurrencia.

    Parámetros:
    ----------
    random_state_dm : qt.Qobj
        Objeto de matriz de densidad cuántica de QuTiP.
    
    desc : int
        Indicador para aplicar la descomposición de Cholesky:
        - Si es 0: no se aplica la descomposición de Cholesky.
        - Si es 1: se aplica la descomposición de Cholesky.

    Retorna:
    --------
    Tuple[List[float], List[float]]
        - Lista de valores reales e imaginarios de la matriz de densidad cuántica.
        - Lista con la concurrencia de la matriz de densidad cuántica.
    
    Excepciones:
    ------------
    - [Error] - Si el valor de `desc` no es válido (0 o 1).
    """
    p_trc_0 = random_state_dm.ptrace([0])
    p_trc_1 = random_state_dm.ptrace([1])
    
    if (desc == 0):
        random_state_dm_arr = np.asarray(random_state_dm)
        diag_elements = random_state_dm_arr.diagonal()
        decomp_rsdm_lower = random_state_dm_arr[np.tril_indices(4, k=-1)]
        decomp_rsdm_lower = decomp_rsdm_lower.flatten()
        
        p_trc_0_dm = np.asarray(p_trc_0)
        p_trc_1_dm = np.asarray(p_trc_1)
        
    elif (desc == 1):
        decomp_rsdm = cholesky(random_state_dm)
        diag_elements = decomp_rsdm.diagonal()
        decomp_rsdm_lower = decomp_rsdm[np.tril_indices(4, k=-1)]
        decomp_rsdm_lower = decomp_rsdm_lower.flatten()
        
        p_trc_0_dm = cholesky(p_trc_0)
        p_trc_1_dm = cholesky(p_trc_1)
    else:
        print('[Error] - Invalid value')
        
    d_ptrc_0 = p_trc_0_dm.diagonal()
    d_ptrc_1 = p_trc_1_dm.diagonal()
    d_ptrc_q0_lower = p_trc_0_dm[np.tril_indices(2, k=-1)]
    d_ptrc_q0_lower = d_ptrc_q0_lower.flatten()
    d_ptrc_q1_lower = p_trc_1_dm[np.tril_indices(2, k=-1)]
    d_ptrc_q1_lower = d_ptrc_q1_lower.flatten()
    
    x_sample = []

    for x in diag_elements:
        x_sample.append(x.real)
    for x in decomp_rsdm_lower:
        x_sample.append(x.real)
    for x in decomp_rsdm_lower:
        x_sample.append(x.imag)
                
    for y in d_ptrc_0:
        x_sample.append(y.real)
    for y in d_ptrc_q0_lower:
        x_sample.append(y.real)
    for y in d_ptrc_q0_lower:
        x_sample.append(y.imag)
        
    for z in d_ptrc_1:
        x_sample.append(z.real)
    for z in d_ptrc_q1_lower:
        x_sample.append(z.real)
    for z in d_ptrc_q1_lower:
        x_sample.append(z.imag)
                        
    conc = qt.concurrence(random_state_dm)
    
    return x_sample, [conc]

# ===========================================
# Proceso principal de generación de estados aleatorios

n_samples = 1000 # Se define una muestra de 1000

columns = ["a11", "a22", "a33", "a44",
           "a21r", "a31r", "a32r", "a41r", "a42r", "a43r",
           "a21i", "a31i", "a32i", "a41i", "a42i", "a43i",
           "ptq0_11", "ptq0_22",
           "ptq0_21r", "ptq0_21i",
           "ptq1_11", "ptq1_22",
           "ptq1_21r", "ptq1_21i",
           "concurrence"]

rs_df_nodesc = pd.DataFrame(columns = columns) #Se crea el dataframe de salida con las columnas definidas
rs_df_desc = pd.DataFrame(columns = columns) #Se crea el dataframe de salida con las columnas definidas

for x in range(n_samples):
    # Genera una matriz de densidad cuántica aleatoria con distribución Ginibre
    random_state_dm = qt.rand_dm_ginibre(4, dims=[[2,2], [2,2]])
    
    x_sample,y_sample = xy2qb(random_state_dm, 0)#Sin descomposición
    xy_sample = x_sample+y_sample
    actual_data_s = pd.Series(xy_sample, index=rs_df_nodesc.columns)
    rs_df_nodesc = rs_df_nodesc.append(actual_data_s, ignore_index=True)
    
    x_sample,y_sample = xy2qb(random_state_dm, 1)#Con descomposición
    xy_sample = x_sample+y_sample
    actual_data_s = pd.Series(xy_sample, index=rs_df_desc.columns)
    rs_df_desc = rs_df_desc.append(actual_data_s, ignore_index=True)

In [ ]:
# ===========================================
# Guardado de conjuntos de datos

rs_df_nodesc.to_csv('./datasets/2qb_random_state_nodesc.csv', index=False, index_label=False)
rs_df_desc.to_csv('./datasets/2qb_random_state_desc.csv', index=False, index_label=False)

# Puros

In [ ]:
# ===========================================
# Import de librerías para configuración de ambiente
import os
import random
print ("Random number with seed 73")
random.seed(73)

os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf_device='/gpu:0'
# ===========================================
# Import de qutip para simulación de la dinámica de sistemas cuánticos abiertos
import qutip as qt
# ===========================================
# Import de librerías para manejo de conjuntos de datos y operaciones matriciales
import pandas as pd

import numpy as np
from numpy.linalg import cholesky

from scipy.stats import unitary_group

from tqdm import tqdm

from typing import Tuple, List

# ===========================================
# Función para generación de n muestras de estados aleatorios de 2 qubits

def xy2qb(random_state_dm: qt.Qobj, desc: int) -> Tuple[List[float], List[float]]:
    """
    Aplica descomposición de Cholesky opcionalmente sobre una matriz de densidad cuántica y 
    retorna una muestra de valores de la matriz junto con su concurrencia.

    Parámetros:
    ----------
    random_state_dm : qt.Qobj
        Objeto de matriz de densidad cuántica de QuTiP.
    
    desc : int
        Indicador para aplicar la descomposición de Cholesky:
        - Si es 0: no se aplica la descomposición de Cholesky.
        - Si es 1: se aplica la descomposición de Cholesky.

    Retorna:
    --------
    Tuple[List[float], List[float]]
        - Lista de valores reales e imaginarios de la matriz de densidad cuántica.
        - Lista con la concurrencia de la matriz de densidad cuántica.
    
    Excepciones:
    ------------
    - [Error] - Si el valor de `desc` no es válido (0 o 1).
    """
    p_trc_0 = random_state_dm.ptrace([0])
    p_trc_1 = random_state_dm.ptrace([1])
    
    if (desc == 0):
        random_state_dm_arr = np.asarray(random_state_dm)
        diag_elements = random_state_dm_arr.diagonal()
        decomp_rsdm_lower = random_state_dm_arr[np.tril_indices(4, k=-1)]
        decomp_rsdm_lower = decomp_rsdm_lower.flatten()
        
        p_trc_0_dm = np.asarray(p_trc_0)
        p_trc_1_dm = np.asarray(p_trc_1)
        
    elif (desc == 1):
        decomp_rsdm = cholesky(random_state_dm)
        diag_elements = decomp_rsdm.diagonal()
        decomp_rsdm_lower = decomp_rsdm[np.tril_indices(4, k=-1)]
        decomp_rsdm_lower = decomp_rsdm_lower.flatten()
        
        p_trc_0_dm = cholesky(p_trc_0)
        p_trc_1_dm = cholesky(p_trc_1)
    else:
        print('[Error] - Invalid value')
        
    d_ptrc_0 = p_trc_0_dm.diagonal()
    d_ptrc_1 = p_trc_1_dm.diagonal()
    d_ptrc_q0_lower = p_trc_0_dm[np.tril_indices(2, k=-1)]
    d_ptrc_q0_lower = d_ptrc_q0_lower.flatten()
    d_ptrc_q1_lower = p_trc_1_dm[np.tril_indices(2, k=-1)]
    d_ptrc_q1_lower = d_ptrc_q1_lower.flatten()
    
    x_sample = []

    for x in diag_elements:
        x_sample.append(x.real)
    for x in decomp_rsdm_lower:
        x_sample.append(x.real)
    for x in decomp_rsdm_lower:
        x_sample.append(x.imag)
                
    for y in d_ptrc_0:
        x_sample.append(y.real)
    for y in d_ptrc_q0_lower:
        x_sample.append(y.real)
    for y in d_ptrc_q0_lower:
        x_sample.append(y.imag)
        
    for z in d_ptrc_1:
        x_sample.append(z.real)
    for z in d_ptrc_q1_lower:
        x_sample.append(z.real)
    for z in d_ptrc_q1_lower:
        x_sample.append(z.imag)
                        
    conc = qt.concurrence(random_state_dm)
    
    return x_sample, [conc]

# ===========================================
# Proceso principal de generación de estados aleatorios

n_samples = 1000 # Se define una muestra de 1000

columns = ["a11", "a22", "a33", "a44",
           "a21r", "a31r", "a32r", "a41r", "a42r", "a43r",
           "a21i", "a31i", "a32i", "a41i", "a42i", "a43i",
           "ptq0_11", "ptq0_22",
           "ptq0_21r", "ptq0_21i",
           "ptq1_11", "ptq1_22",
           "ptq1_21r", "ptq1_21i",
           "concurrence"]

#Se crea el dataframe de salida con las columnas definidas
rs_df_nodesc = pd.DataFrame(columns = columns) 
#Se crea el dataframe de salida con las columnas definidas
rs_df_desc = pd.DataFrame(columns = columns)

for x in tqdm(range(n_samples)):
    # Genera una matriz unitaria aleatoria 4x4
    U = unitary_group.rvs(4)
    # Calcula la matriz conjugada transpuesta de U
    U_D = U.conj().T
    # Se define una matriz M_D, que corresponde a un estado puro
    # (un solo qubit en el estado base |0⟩)
    M_D = [[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
    #-------------------------------------------------------------
    # Calcular el estado rho_p aplicando la matriz unitaria a M_D
    # rho_p = U† * M_D * U
    rho_p = np.dot(U_D, M_D)
    rho_p = np.dot(rho_p, U)
    # Crear el objeto Qobj de QuTiP a partir de rho_p
    # dims=[[2,2], [2,2]] especifica que la matriz representa un sistema de dos qubits
    random_state_dm = qt.Qobj(rho_p, dims=[[2,2], [2,2]])
    
    x_sample,y_sample = xy2qb(random_state_dm, 0)# Sin descomposición
    xy_sample = x_sample+y_sample # Se combinan los resultados de x_sample e y_sample
    # Crear una nueva fila para el DataFrame con los datos generados y agregarla a rs_df_nodesc
    actual_data_s = pd.Series(xy_sample, index=rs_df_nodesc.columns)
    rs_df_nodesc = rs_df_nodesc.append(actual_data_s, ignore_index=True)

In [ ]:
# ===========================================
# Guardado de conjuntos de datos

rs_df_nodesc.to_csv('./datasets/2qb_random_state_nodesc_pure_1000.csv', index=False, index_label=False)

# Mezclados 50% Puros - 50% Mezclados

In [ ]:
# ===========================================
# Import de librerías para configuración de ambiente
import os
import random
print ("Random number with seed 73")
random.seed(73)

os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf_device='/gpu:0'
# ===========================================
# Import de librerías para manejo de conjuntos de datos y operaciones matriciales
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
# ===========================================
# Carga de datasets anteriores

nopure_df = pd.read_csv('./datasets/2qb_random_state_nodesc.csv')
pure_df = pd.read_csv('./datasets/2qb_random_state_nodesc_pure_1000.csv')
# ===========================================
n = 500

nopure_df_sample = nopure_df.iloc[:n]
nopure_df_sample['dataset'] = 'A'

pure_df_sample = pure_df.iloc[:n]
pure_df_sample['dataset'] = 'B'
#---------------------------
frames = [nopure_df_sample, pure_df_sample]
  
mix_df = pd.concat(frames)
mix_df.reset_index(drop=True, inplace=True)

#Se calcula la cantidad de datos para train, val y test, siguiendo regla 70/20/10 respectivamente
trn_data_count = int(len(mix_df)*0.7)
val_data_count = int(len(mix_df)*0.2)
tst_data_count = int(len(mix_df)*0.1)

print(trn_data_count, val_data_count, tst_data_count)

Xs = mix_df[mix_df.columns[:-10]]
Xs['conc'] = mix_df['concurrence']
Ys = mix_df[mix_df.columns[-1:]]
#---------------------------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(Xs, Ys, stratify=Ys, test_size=0.30, random_state=73)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, stratify=y_test, test_size=0.333, random_state=73)
#---------------------------------------------------------------------------------------
mix_x_df = [X_train, X_val, X_test]
mix_x_df = pd.concat(mix_x_df).reset_index()
mix_y_df = [y_train, y_val, y_test]
mix_y_df = pd.concat(mix_y_df).reset_index(drop=True)

mix_full_new = pd.concat([mix_x_df, mix_y_df], axis=1)

# ===========================================
# Guardado de conjuntos de datos

mix_full_new.to_csv('./datasets/2qb_random_state_50A_50B.csv')